# Архитектура Residual Net

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split, DataLoader

In [13]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = CIFAR10(root="./data", download=True, transform=transform)

train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

In [14]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        return self.relu(out)


In [15]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(ResBlock, 64, 64, blocks=2, stride=1)
        self.layer2 = self._make_layer(ResBlock, 64, 128, blocks=2, stride=2)
        self.layer3 = self._make_layer(ResBlock, 128, 256, blocks=2, stride=2)
        self.layer4 = self._make_layer(ResBlock, 256, 512, blocks=2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [16]:
epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

patience = 1
best_val_loss = float("inf")
patience_counter = 0
model_path = "best_resnet_classification.pth"

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct_train = 0
  total_train = 0

  for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    preds = torch.argmax(outputs, dim=1)
    correct_train += (preds == labels).sum().item()
    total_train += labels.size(0)

  train_loss = running_loss /  len(train_loader)
  train_accuracy = correct_train / total_train

  model.eval()
  val_loss = 0.0
  correct_val = 0
  total_val = 0

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      preds = torch.argmax(outputs, dim=1)
      correct_val += (preds == labels).sum().item()
      total_val += labels.size(0)

  val_loss /= len(val_loader)
  val_accuracy = correct_val / total_val


  print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    patience_counter = 0
    torch.save(model.state_dict(), model_path)
    print(f"✔ Model improved. Saved to {model_path}")
  else:
    patience_counter += 1
    if patience_counter >= patience:
      print("Ранняя остановка сработала!")
      break

model.load_state_dict(torch.load(model_path))
model.to(device)
print(f"✅ Best model loaded from {model_path}")

model.eval()
test_loss = 0.0
correct_test = 0
total_test = 0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    test_loss += loss.item()

    preds = torch.argmax(outputs, dim=1)
    correct_test += (preds == labels).sum().item()
    total_test += labels.size(0)

test_loss /= len(test_loader)
test_accuracy = correct_test / total_test
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/1: 100%|██████████| 469/469 [08:15<00:00,  1.06s/it]


Epoch 1, Train Loss: 1.4775, Val Loss: 1.2793
✔ Model improved. Saved to best_resnet_classification.pth
✅ Best model loaded from best_resnet_classification.pth
Test Loss: 1.2864, Test Accuracy: 0.5464


In [18]:
from google.colab import files
files.download("best_resnet_classification.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>